# **Import Dataset and Libraries**

In [1]:
!pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
! kaggle datasets download benhamner/sf-bay-area-bike-share

Dataset URL: https://www.kaggle.com/datasets/benhamner/sf-bay-area-bike-share
License(s): unknown
 99% 551M/554M [00:11<00:00, 42.9MB/s]
100% 554M/554M [00:11<00:00, 50.1MB/s]


In [ ]:
# get crime data from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install pyspark

In [ ]:
!unzip /content/sf-bay-area-bike-share.zip -d /content

Archive:  /content/sf-bay-area-bike-share.zip
  inflating: /content/database.sqlite  
  inflating: /content/station.csv    
  inflating: /content/status.csv     
  inflating: /content/trip.csv       
  inflating: /content/weather.csv    


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
from matplotlib import rcParams

import folium
from folium import plugins
from folium.plugins import HeatMap

import plotly.express as px

from geopandas import GeoDataFrame # Loading boundaries Data

from shapely.geometry import Point, Polygon, shape # creating geospatial data
from shapely import wkb, wkt # creating and parsing geospatial data

from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, round, to_timestamp, hour, dayofweek, count, month
from pyspark.sql.functions import radians, sqrt, sin, cos, asin, udf, lit, rank, expr
from pyspark.sql.functions import *

from pyspark.sql.types import StringType, DateType, FloatType, IntegerType
from pyspark.sql.window import Window

from collections import Counter

from ast import literal_eval as make_tuple # used to decode data from java

import lightgbm as lgb
from sklearn.model_selection import KFold

import requests

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# **Spark Session and Loading Datasets**

In [ ]:
spark = SparkSession.builder.master("local").appName("BigDataProject").config('spark.ui.port','4050').getOrCreate()

In [ ]:
statusdf = spark.read.csv('/content/drive/MyDrive/UCR/226_Big_Data/Project/Dataset/status.csv', header=True, inferSchema=True)
stationdf = spark.read.csv('/content/drive/MyDrive/UCR/226_Big_Data/Project/Dataset/station.csv', header=True, inferSchema=True)
tripdf = spark.read.csv('/content/drive/MyDrive/UCR/226_Big_Data/Project/Dataset/trip.csv', header=True, inferSchema=True)
crimedf = spark.read.csv('/content/drive/MyDrive/UCR/226_Big_Data/Project/Dataset/crime_report.csv', header=True, inferSchema=True)

In [ ]:
statusdf.show(5)
statusdf.count()

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|              2|             25|2013/08/29 12:06:01|
|         2|              2|             25|2013/08/29 12:07:01|
|         2|              2|             25|2013/08/29 12:08:01|
|         2|              2|             25|2013/08/29 12:09:01|
|         2|              2|             25|2013/08/29 12:10:01|
+----------+---------------+---------------+-------------------+
only showing top 5 rows



71984434

In [ ]:
stationdf.show(5)
stationdf.count()

+---+--------------------+------------------+-------------------+----------+--------+-----------------+
| id|                name|               lat|               long|dock_count|    city|installation_date|
+---+--------------------+------------------+-------------------+----------+--------+-----------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|San Jose|         8/5/2013|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|San Jose|         8/6/2013|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|San Jose|         8/5/2013|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|        15|San Jose|         8/7/2013|
+---+--------------------+------------------+-------------------+----------+--------+-----------------+
only showing top 5 rows



70

In [ ]:
crimedf.show(5)
crimedf.count()

+--------------------+-------------+-------------------+-------------+--------------------+--------------------+------------+-----------+---------------+----------+----------------+-----------------------+------------+-------------+-----------------+--------------------+--------------------+--------------+------------+----+---------------+---------------------+-------------------+------------------------+--------+---------+-----+-------------+----------------------+----------------------------------------------------+--------------------------------------------+---------------------------+-----------------------------------+----------------------------+------------------------+
|   Incident Datetime|Incident Date|      Incident Time|Incident Year|Incident Day of Week|     Report Datetime|      Row ID|Incident ID|Incident Number|CAD Number|Report Type Code|Report Type Description|Filed Online|Incident Code|Incident Category|Incident Subcategory|Incident Description|    Resolution|Inters

798476

In [ ]:
tripdf.show(5)
tripdf.count()

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|8/29/2013 14:13|South Van Ness at...|              66|8/29/2013 14:14|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|      70|8/29/2013 14:42|  San Jose City Hall|              10|8/29/2013 14:43|  San Jose City Hall|            10|    661|       Subscriber|   95138|
|4130|      71|8/29/2013 10:16|Mountain View Cit...|              27|8/29/2013 10:17|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|      77|8/29/2013 11:29|  San Jose City Hall|      

669959

In [ ]:
tripdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)



# **Data Cleaning**

In [ ]:
#remove NaN values from crime dataframe
print(crimedf.count())
crimedf = crimedf.dropna(subset=['Latitude', 'Longitude'])
crimedf.count()

798476


755095